<a href="https://colab.research.google.com/github/DaveWates/Assignment-ISYS5002/blob/main/Assignment3_lecturer_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This Program determines the salary of key executive personnel of companies whereby a user  provides a stock market ticker. (I have chosen the major Australian Banks as a test sample). It will then calculate the tax payable on the reported income.** 

**The ticker is input in Yahoo Finance link https://finance.yahoo.com/?fr=sycsrp_catchall where the URL link for the company profile page may be sourced.** 

**1. Load the packages**


In [ ]:
!pip install kora -q
from kora.selenium import wd
from bs4 import BeautifulSoup

**2.Prompt the user for the ticker and locate the URL from Yahoo Finance.**

In [ ]:
#Prompt user for Bank
bank = input('Which bank do you want to search?;choose from ANZ, CBA, NAB, WBC = ')

urls = ('https://finance.yahoo.com/quote/ANZ.AX/profile?p=ANZ.AX',
       'https://finance.yahoo.com/quote/CBA.AX/profile?p=CBA.AX',
       'https://finance.yahoo.com/quote/NAB.AX/profile?p=NAB.AX',
       'https://finance.yahoo.com/quote/WBC.AX/profile?p=WBC.AX')

def get_url():
  for url in urls:
   if bank == 'ANZ':
    return (urls[0])
   if bank == 'CBA':
    return (urls[1])
   if bank == 'NAB':
    return (urls[2])
   if bank == 'WBC':
    return (urls[3])

print('The url required is', get_url())



Which bank do you want to search?;choose from ANZ, CBA, NAB, WBC = WBC
The url required is https://finance.yahoo.com/quote/WBC.AX/profile?p=WBC.AX


# LECTURER NOTE #1

Having a list of URL, and then checking for the string to get to the index is good logic.  But for this assignment is for Australia companies, not just the banks.

So you will need to do something like we did for Amazon.  With the amazon example we usead a search term, in this example we ask the user for a ticker.

    ticker = input("What company do you want me to search for? ")
    url = get_url(ticker)

and get_url() is something like:

    def get_url(thing):
      template = 'https://finance.yahoo.com/quote/{}/profile?p={}'
      url = template.format(thing, thing)
      return url

The function uses a format string.  The way to use a format string is to create a string and use '{}' as a placeholder to identify where you want to update the string by replacing the '{}'.   I can then call the format() method and give it value to be used.  In this case there are two sets of '{}' and they both need to be the same thing soe we do something like:

    template.format(thing, thing)

Notice how we gave the format() function two values, one for each set of '{}' o the string.


**3. Use the related URL to scrape the company profile page.**

In [ ]:
wd.get(get_url())
soup = BeautifulSoup(wd.page_source, 'html.parser')
tables = soup.find_all('table')
for table in tables:
  body = table.find('tbody')
  rows = body.find_all('tr')
  for row in rows:
    data = row.find_all('td')
    for txt in data:
      print(txt.text)
    print()

# Lecturer Note #2

The following code (from above) will print out the data in each cell of each table 

    wd.get(get_url())
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    tables = soup.find_all('table')
    for table in tables:
      body = table.find('tbody')
      rows = body.find_all('tr')
      for row in rows:
        data = row.find_all('td')
        for txt in data:
          print(txt.text)
        print()

For this assignment, yahoo finance only has one table.  The information we need is the executives name, title and pay.  Looking at the table from Yahoo, we see the name is hte firxt column/cell, title is the second column/cell and the pay is in the third column/cell.

Now find_all() wil return a list of table, and we want the first table so to get the table we need something like:

    tables = soup.find_all('table')
    table = tables[0]. # Only want the fist table

But with BeautifulSoup, if we only want the 'first' one of something we can use the find() function.  So the follwing also find the first table:

    table = soup.find('table')

It can be a little bit confusing, and sometimes we dont notice the difference between the functions 'find_all()' and 'find()'

Either method is OK, so long as you get the table.  So the code becomes:

    wd.get(get_url())
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    table = soup.find('table')
    body = table.find('tbody')
    rows = body.find_all('tr')
    for row in rows:
      data = row.find_all('td')
      for txt in data:
        print(txt.text)
      print()

We can improve it further, for each row, we know we only want the first three cell, name, title and pay.  It would be great if we could turn the row into a record:

    name: John Smith 
    title: CEO
    pay: 5.4M

This looks a bit like a dictionary in Python.  We have a key, eg. name, and a value 'John Smith'.   The key we create, and the value we read form the table. 
The 'name' if the first cell/column, title is the second cell/column and the pay is the third cell/column. So the code becomes:

    wd.get(get_url())
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    table = soup.find('table')
    body = table.find('tbody')
    rows = body.find_all('tr')
    for row in rows:
      data = row.find_all('td')
      # Remeber computer start counting form zero, so the 'first' is '0'
      record = {'name': data[0], 'title': data[1], 'pay': data[2]}

So we can create records as a python dictionary.  Okay, so that gets us one record, but how can I save every row.  Well we need to create a list to to store/save each record.  Each time we create a new record, we add/append it to the list.  

    wd.get(get_url())
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    table = soup.find('table')
    body = table.find('tbody')
    rows = body.find_all('tr')
    list_of_records = []. # create a empty list to save/store our records
    for row in rows:
      data = row.find_all('td')
      # Remeber computer start counting form zero, so the 'first' is '0'
      record = {'name': data[0], 'title': data[1], 'pay': data[2]}
      list_of_records.append(record)  # save the record to our list
    # lets see if that workd, print out the records
    print(str(list_of_records))

So we have a list of records, well actually a 'list of dictionaries' since each record is actually a python dictionary.  Now if we are happy that the code works, lets put the record creation into a function

    def get_records(soup):
      table = soup.find('table')
      body = table.find('tbody')
      rows = body.find_all('tr')
      list_of_records = []. # create a empty list to save/store our records
      for row in rows:
        data = row.find_all('td')
        # Remeber computer start counting form zero, so the 'first' is '0'
        record = {'name': data[0], 'title': data[1], 'pay': data[2]}
        list_of_records.append(record)  # save the record to our list
      
      return records


It not quite as simple.  We know later that we need to calculate the tax payable. To do this we will need a number, but the pay data form the table is something like: 5.4M, or 929.8k and we need to convert this to: 5400000 and 939800.

This can be tricky, but will give some hints.
Lets assume pay = '5.4M'.  This is a string. In python a string is a list of characters.  Being a list, I can use a index to get to characters.  I can get the last character by using a '-1' index.

    last_char = pay[-1]

I can strip or remove the last character, but copying everything upto but no including the last character

    pay_number = pay[0:-1] # get first, to one less

This should be an number, since it has a decimal point it data type is a float

    pay_number = float(pay[0:-1])

If the 'M' we need to multiply by 1000000, if its is 'k' we multiple 1000 and if it is a 'A' then that means no pay values was entered in the table.

    num_part = pay[0:-1]
    if last_char == 'M':
        salary = float(num_part) * 1000000
    elif last_char == 'k': 
        salary = float(num_part) * 1000
    else:
        salary = 0

That is a little confusing, let put it into a function:

    def get_salary(salary_string):
       last_char = salary_string[-1] 
       num_part = salary_string[0:-1]
        if last_char == 'M':
            salary = float(num_part) * 1000000
        elif last_char == 'k': 
            salary = float(num_part) * 1000
        else: # Must a N/A entry
            salary = 0
       return salary


So we get records, we get something like:

    def get_records(soup):
      table = soup.find('table')
      body = table.find('tbody')
      rows = body.find_all('tr')
      list_of_records = []. # create a empty list to save/store our records
      for row in rows:
        data = row.find_all('td')
        # Remeber computer start counting form zero, so the 'first' is '0'
        pay = get_salary(data[2])
        record = {'name': data[0], 'title': data[1], 'pay': pay}
        list_of_records.append(record)  # save the record to our list
      
      return records




And our code becomes

    wd.get(get_url())
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    executives = get_records(soup)


Now to make thing easier, we should put this into a panadas dataframe. In the Amazon example, I showed how to create a dataframe from a list of dictoinaries.  Or you could google 'How do I create a dataframe form a list of dictionaries'

    import panadas as pd
    executivies_df = pd.from_records(executives)

Now one everyting is in a dataframe, we can work out how to add a column to calculate the tax.  **See Lecture Note #3**


4. **Manipulate the data for ease of reference using Pandas.**

In [ ]:
import pandas as pd


In [ ]:
# Creating a data frame df.
df = pd.DataFrame({'NAME':['Peter Francis King', 'Michael  Rowland', 'Leslie Wilson Vance', 'Rebecca  Lim','Christine  Parker','Jason  Yetton','Carolyn  McCann','David  Stephen'],'SALARY':[2810000, 1220000, 1000000, 871920,999800,1250000,941000,1890000]})
# Head displays only the executives with reported income from the data frame.
df.head(10)

,NAME,SALARY
0,Peter Francis King,2810000
1,Michael Rowland,1220000
2,Leslie Wilson Vance,1000000
3,Rebecca Lim,871920
4,Christine Parker,999800
5,Jason Yetton,1250000
6,Carolyn McCann,941000
7,David Stephen,1890000


**5. Determine tax payable on the reported income using the ATO tax rate table.**

[Resident tax rates 2021–22
Taxable income](https://www.ato.gov.au/rates/individual-income-tax-rates/)

Income            | Tax on this income 
------------------|----------------------
0 – \$18,200       | Nil
\$18,201 – \$45,000 | 19 cents for each \$1 over \$18,200
\$45,001 – \$120,000 | \$5,092 plus 32.5 cents for each \$1 over \$45,000
\$120,001 – \$180,000 |  \$29,467 plus 37 cents for each \$1 over \$120,000
\$180,001 and over | \$51,667 plus 45 cents for each \$1 over \$180,000



In [ ]:
# A formula to calculate tax payable based on the above rates

In [ ]:
def calc_tax(income):
  if income <= 18200:
    tax = 0
  elif 18200 < income <= 45000:
    tax = (income - 18200) * 0.19
  elif 45000 < income <= 120000:
    tax = ((income - 45001) * 0.325) + 5092
  elif 120000 < income <= 180000:
    tax = ((income - 120001) * 0.37) + 29467
  elif 18000 < income: 
    tax = ((income - 180001) * 0.45) + 51667
  return tax

In [ ]:
# Match Executive's name with reported salary

name = input('Please enter the Executives name from the list above  = ')
incomes= ('2810000','1220000','1000000','871920','999800','1250000','941000','1890000')

def get_salary_from_name():
  for income in incomes:
    if name == 'Peter Francis King':
     return (incomes[0])
    if name == 'Michael Rowland':
     return (incomes[1])
    if name == 'Leslie Wilson Vance':
     return (incomes[2])
    if name == 'Rebecca Lim':
     return (incomes[3]) 
    if name == 'Christine Parker':
     return (incomes[4])
    if name == 'Jason Yetton':
     return (incomes[5])
    if name == 'Carolyn McCann':
     return (incomes[6])
    if name == 'David Stephen':
     return (incomes[7])

print("The taxable salary is", "$",get_salary_from_name())
income = int(get_salary_from_name())
tax = calc_tax(income)
print ("The tax payable on that income is","$", tax)

Please enter the Executives name from the list above  = David Stephen
The taxable salary is $ 1890000
The tax payable on that income is $ 821166.55


# Lecturer Note #3

You are on the right track.  Hopefully the follwing will help.

Now  assuming you got the above working, we have a dataframe called 'executives.df'. This contains name, title and pay of each executive.  Now we need to add the tax.

The reason I recommend panads, is kinda like a excell spread sheet.  Pandas provides lots of useful functions to work on data.

In a dataframe, a column is identified by its column_name.

So "executivies_df['name']" will give me all the names in the table.

To add a new column, we just create a new 'key', so if I want a column with tax_payable I do the following:

    executivies_df['tax_payable'] = something_to_work_out

I will try to help with the 'something_to_work_out'

The general logic is

    for each executive's pay
      claculate the tax and add to the table

I see to have this general logic above.  SO if we can loop over the each row in the executives salary and calculate the tax.  Let use a similar pattern form before, create a empty list, calculat the tax and add to the list.

    tax_payable = []
    for salary in executives_df['salary']:
      tax = calc_tax(salary)
      tax_payable.append(tax)

Now to add the new colume we simply let is equal the list we just created.

    executives_df['tax_payable'] = tax_payable


**I hope that helps**